In [5]:
import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 250 kB 2.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
# sheet name test
""" import pandas as pd

# Load the uploaded Excel file to examine its structure
file_path = '../../species-portraits/portraits/20230619_Gimpel_Pyrrhula_Pyrrhula_BP_AM.xlsx'
xls = pd.ExcelFile(file_path)

# Checking sheet names to understand the file's structure
sheet_names = xls.sheet_names
sheet_names
"""

['Portrait', 'Lebenszyklus', 'Pflanzenliste']

In [8]:
# sheet correctness check
# portrait_df = pd.read_excel(xls, sheet_name='Pflanzenliste')
# portrait_df.head()

,Kategorie,Art_botanisch,Art_deutsch,Zweck,Quelle,Anmerkungen
0,Baum,Acer campestris,Ahorn,"Nahrung (Samen, Knospen)",NaN,NaN
1,"Baum, Hecke",Acer campestris,Ahorn,Schutz- und Nistgehölz,NaN,NaN
2,Einjährige,Alliaria petiolata,Knoblauchsrauke,Nahrung (Samen),NaN,NaN
3,Staude,Artemisia vulgaris,Gewöhnlicher Beifuß,Nahrung (Samen),Glutz von Blotzheim 2001,NaN
4,Baum,Betula pendula,Hänge-Birke,"Nahrung (Kätzchen, Samen)",NaN,NaN


In [3]:
import pandas as pd
import os

# Verzeichnis, in dem sich alle Excel-Dateien befinden
directory = '../../species-portraits/portraits'

# Definiere die Zielspeicherort für die CSV-Dateien
output_directory = './out/relations'

# Lege sicher, dass der Ordner existiert, ansonsten erstelle ihn
os.makedirs(output_directory, exist_ok=True)

# Alle Dateien im Verzeichnis durchgehen
for filename in os.listdir(directory):
    # Überprüfen, ob die Datei eine .xlsx-Datei ist
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        
        # Excel-Datei laden und das Sheet "Pflanzenliste" einlesen
        try:
            df_plants = pd.read_excel(file_path, sheet_name='Pflanzenliste')
        except Exception as e:
            print(f"Fehler beim Laden der Datei {filename}: {e}")
            continue

        # Überprüfen, ob die notwendigen Spalten vorhanden sind
        required_columns = ['scientific_name', 'Zweck', 'Anmerkungen', 'Quelle']
        if not all(col in df_plants.columns for col in required_columns):
            print(f"Erforderliche Spalten fehlen in {filename}")
            continue

        # Artenportrait aus dem Arbeitsblatt "Artenportrait" laden
        try:
            df_species = pd.read_excel(file_path, sheet_name='Portrait')
            species_name = df_species.loc[0, 'Text']  # Annahme: Der lateinische Name steht in der ersten Zeile unter 'Text'
        except Exception as e:
            print(f"Fehler beim Laden des Arbeitsblatts 'Artenportrait' in {filename}: {e}")
            continue
        
        print(species_name)
        # Beziehungstabelle erstellen (species, plant, purpose, annotations, sources)
        df_relationship = pd.DataFrame()
        #df_relationship['species'] = species_name
        df_relationship['plant'] = df_plants['scientific_name']  # Die Pflanzenart
        df_relationship['purpose'] = df_plants['Zweck']  # Nutzungszweck
        df_relationship['annotations'] = df_plants['Anmerkungen']  # Anmerkungen
        df_relationship['sources'] = df_plants['Quelle']  # Quellen

        df_relationship.insert(0, 'species', species_name)

        # Hinzufügen einer 'id'-Spalte mit fortlaufenden Ganzzahlen
        df_relationship.insert(0, 'id', range(1, len(df_relationship) + 1))

        print(df_relationship.head())
        
        # Zielpfad für die CSV
        #output_csv_path = os.path.join(output_directory, f"{filename.split('.')[0]}_species_plant_relationship.csv")
        output_csv_path = os.path.join(output_directory, f"{species_name}_species_plant_relationship.csv")
        
        # CSV-Datei exportieren
        df_relationship.to_csv(output_csv_path, index=False)

        print(f'Die Beziehungstabelle für {filename} wurde erfolgreich exportiert nach: {output_csv_path}')


Andrena barbilabris
   id              species                  plant                   purpose  \
0   1  Andrena barbilabris         Acer campestre  Pollen- und Nektarquelle   
1   2  Andrena barbilabris    Acer pseudoplatanus  Pollen- und Nektarquelle   
2   3  Andrena barbilabris  Aegopodium podagraria  Pollen- und Nektarquelle   
3   4  Andrena barbilabris  Antrhiscus sylvestris  Pollen- und Nektarquelle   
4   5  Andrena barbilabris         Brassica napus  Pollen- und Nektarquelle   

                         annotations        sources  
0                                NaN  Westrich 2018  
1                                NaN  Westrich 2018  
2                                NaN  Westrich 2018  
3                                NaN  Westrich 2018  
4  beliebte Nektar- und Pollenquelle  Westrich 2018  
Die Beziehungstabelle für 20241105_Bärtige Sandbiene_Andrena barbilabris_AM.xlsx wurde erfolgreich exportiert nach: ./out/relations/Andrena barbilabris_species_plant_relationship.c